In [1]:
import h5py
import pandas as pd
import numpy as np
import paho.mqtt.publish as publish
import paho.mqtt.subscribe as subscribe

In [2]:
#Sequence length
seq_len = 10

filename = 'N-CMAPSS_DS03-012.h5'

dataset = filename[-11:-3]

In [3]:
with h5py.File(filename, 'r') as hdf:
    
    X_test = pd.concat([pd.DataFrame(np.array(hdf.get(f'W_test')), columns=np.array(hdf.get('W_var')).astype('U')),
                pd.DataFrame(np.array(hdf.get('X_s_test')), columns=np.array(hdf.get('X_s_var')).astype('U'))], axis=1)
    
    X_test.rename(columns={'alt':'Altitude'}, inplace=True)
    
    Xw_columns = X_test.shape[1]
    
    aux_test = pd.concat([pd.DataFrame(np.array(hdf.get('Y_test')), columns=['RUL']),
                pd.DataFrame(np.array(hdf.get('A_test')), columns=np.array(hdf.get('A_var')).astype('U')).astype('int64')], axis=1)          

    aux_test.rename(columns={'unit':'Unit','cycle':'Cycle','Fc':'Class','hs':'State'}, inplace=True)
    
    aux_columns = aux_test.shape[1]
    
    test_units = aux_test['Unit'].unique()[len(aux_test['Unit'].unique())//2:]
    
    # Compute length inital two datasets
    train_len = len(np.array(hdf.get('X_s_dev')))
    test_len = len(X_test)
    total_len = train_len + test_len
    
    test = pd.concat([aux_test, X_test], axis=1)

    #Update the test set
    X_test = test[test['Unit'].isin(test_units)].iloc[:, -Xw_columns:].reset_index(drop=True)
    aux_test = test[test['Unit'].isin(test_units)].iloc[:, :aux_columns].reset_index(drop=True)
    text_units = aux_test['Unit'].unique()

    del test

    # Compute length dataset
    test_len = len(X_test)

    
    print(f'The test set:  {round(test_len/total_len*100)}%  {test_len:,} observations. Units: {test_units}'.replace(',','.'))

The test set:  21%  2.070.550 observations. Units: [13 14 15]


In [4]:
test = pd.concat([aux_test, X_test], axis=1)

In [5]:
def wait_for_request(client, userdata, msg):
    
    global unit, cycle, idx

    cycle_data = test[(test['Unit'] == unit) & (test['Cycle'] == cycle)]
    
    # If the n. of cycles is the last, remove unit and go to the next one
    if not len(cycle_data):
        
        idx += 1
        
        # If units are finished, restart
        if idx == len(test_units):     
            idx = 0
        
        
        cycle = 1
        unit = test_units[idx]
        cycle_data = test[(test['Unit'] == unit) & (test['Cycle'] == cycle)]
        
    
    publish.single("diagnostics", payload=cycle_data.to_json(), qos=1, hostname="localhost")
    
    cycle += 1


In [ ]:
# Setting initial unit and cycle
idx = 0
cycle = 1
unit = test_units[idx]

try:
    subscribe.callback(wait_for_request, "request", qos=1, hostname="localhost")
except ConnectionRefusedError:
    print('Broker may not be available')